In [1]:
from transformers import BertTokenizer
import random
import torch
from transformers import BertForSequenceClassification, BertTokenizer
import pickle
import time
import sys
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import json
from nltk.corpus import stopwords

In [2]:
# Load the trained model and tokenizer
model = BertForSequenceClassification.from_pretrained("./model")
tokenizer = BertTokenizer.from_pretrained("./model")

# Move the model to the appropriate device (e.g., GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Set the model to evaluation mode
model.eval()

# Initialize the WordNet Lemmatizer
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
# Load the fitted LabelEncoder
with open("label_encoder.pkl", "rb") as f:
    encoder = pickle.load(f)

In [3]:
def greet_user():
    greetings = ["Hi! How can I assist you today?", "Hello! How can I help you today?"]
    return random.choice(greetings)

def get_treatment(user_input):
    if "treatment" in user_input.lower() or "cure" in user_input.lower() or "treatments" in user_input.lower() or "cures" in user_input.lower() or "what are the treatments" in user_input.lower():
        return True
    else:
        return False

def possible_disease_enquiry(user_input):
    enquiry_words = ["possible disease","possible diseases","chances","chance","predicted disease","predicted illness","scope","predict"]
    return any(word in user_input.lower() for word in enquiry_words)

def is_exit_message(user_input):
    exit_words = ["thank", "thanks", "goodbye", "bye", "quit"]
    return any(word in user_input.lower() for word in exit_words)

def preprocess_text(text):
    words = nltk.word_tokenize(text)
    words = [word for word in words if word not in stop_words]
    words = [lemmatizer.lemmatize(word) for word in words]
    return " ".join(words)

def extract_symptoms(input_text):
    word_list = nltk.word_tokenize(input_text)
    symptoms = [lemmatizer.lemmatize(word.lower()) for word in word_list if word not in stop_words]
    return symptoms

def predict_intent(user_input):
    treatmentss = []
    encoded_inputs = tokenizer(user_input, truncation=True, padding=True, return_tensors="pt")
    input_ids = encoded_inputs["input_ids"].to(device)
    attention_mask = encoded_inputs["attention_mask"].to(device)
    # Perform classification
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predicted_labels = torch.argmax(logits, dim=1)
    # Convert predicted labels to original classes using the LabelEncoder
    predicted_classes = encoder.inverse_transform(predicted_labels.cpu().numpy())
    # Get the predicted disease
    predicted_disease = predicted_classes[0]
    for intent in disease_data["intents"]:
        if intent["name"].lower() == predicted_disease.lower():
            treatments = intent["treatments"]
            if treatments:
                for treatment in treatments:
                    treatmentss.append(treatment)
    for intent in disease_data["intents"]:
        if intent["name"].lower() == predicted_disease.lower():
            description = intent["description"]
    max_proba = torch.max(torch.softmax(logits, dim=1)).item()
    return description, treatmentss, predicted_disease, max_proba

def get_possible_diseases(user_input):
    symptoms = extract_symptoms(user_input)
    possible_diseases = []

    for intent in disease_data["intents"]:
        for symptom in symptoms:
            if symptom in intent["symptoms"]:
                possible_diseases.append(intent["name"])
                break

    return possible_diseases

def get_possible_diseases_and_treatments(user_input):
    symptoms = extract_symptoms(user_input)
    possible_diseases = []
    treatments = {}
    w, x, y, z = predict_intent(user_input)

    for intent in disease_data["intents"]:
        for symptom in symptoms:
            if symptom in intent["symptoms"]:
                possible_diseases.append(intent["name"])
                treatments[intent["name"]] = intent["treatments"]
                break

    return y,possible_diseases, x,treatments

def resetting_user_input(user_input):
    if "another symptoms" in user_input.lower() or "help me with another set of symptoms" in user_input.lower() or "new set" in user_input.lower() or "new chat" in user_input.lower() or "fresh" in user_input.lower() or "start" in user_input.lower() or "from beginning" in user_input.lower():
        return True
    else:
        return False

def typing_effect(message):
    for char in message:
        sys.stdout.write(char)
        sys.stdout.flush()
        time.sleep(0.05)
    print("") 

In [4]:
# Load the disease data
with open("data.json", "r") as f:
    disease_data = json.load(f)

In [5]:
typing_effect(greet_user())
symptoms_input = ""
possible_diseases = []
possible_treatments = {}
# Confidence threshold
confidence_threshold = 0.12
while True:
    # User enters a symptom
    user_input = input("User: ")
    
    if resetting_user_input(user_input):
        symptoms_input = ""
        possible_diseases = []
        possible_treatments = {}
        typing_effect("Sure, how can I help you")
        continue
       
    if is_exit_message(user_input):
        typing_effect("Chatbot: You're welcome! Take care!")
        break

    # Add symptom to user's symptoms
    symptoms_input += " " + user_input
    description, treatments, predicted_disease, confidence = predict_intent(symptoms_input)
    
    if confidence < confidence_threshold:
        if possible_disease_enquiry(user_input):
            y, possible_diseases,x, possible_treatments = get_possible_diseases_and_treatments(symptoms_input)
            if len(possible_diseases)>0:
                typing_effect(f"Chatbot: The possible diseases based on your symptoms are: {y}, {', '.join(possible_diseases)}")
            else:
                typing_effect("No other predictions as per my knowledge")
            continue    

        if get_treatment(user_input):
            if len(possible_diseases) > 0:
                typing_effect(f"Chatbot: The treatments for {y} are: {x}")
                for disease in possible_diseases:
                    typing_effect(f"Chatbot: The treatments for {disease} are: {', '.join(possible_treatments[disease])}")
            else:
                typing_effect("Chatbot: You haven't asked for possible diseases yet. Please ask for possible diseases first.")
            continue
            
        common_symptoms_response = ["Your symptoms are quite common, could you please provide more details or any other symptom?", 
                                   "That is a very common symptom, provide me with another symptom you are facing","That's a quite common symptom for many diseases, provide me with any another symptom you are having?"]
        sol = random.choice(common_symptoms_response)
        typing_effect(f"Chatbot: {sol}")
        typing_effect(f"Confidence: {confidence}")
        continue
        
    for item in description:
        print(*item, end="")
    print()
        
    typing_effect(f"Confidence: {confidence}")
    
    user_input = input("User: ")
    
    if get_treatment(user_input):
        typing_effect(f"Chatbot: The treatments are: {treatments}")
        continue
    
    if resetting_user_input(user_input):
        symptoms_input = ""
        possible_diseases = []
        possible_treatments = {}
        typing_effect("Sure, how can I help you")
        continue      
        
    if is_exit_message(user_input):
        typing_effect("Chatbot: You're welcome! Take care!")
        break    

Hi! How can I assist you today?
User: I have diarrhea and vomiting


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Chatbot: That's a quite common symptom for many diseases, provide me with any another symptom you are having?
Confidence: 0.03952980414032936
User: I have nausea
Chatbot: That's a quite common symptom for many diseases, provide me with any another symptom you are having?
Confidence: 0.058688681572675705
User: thanks
Chatbot: You're welcome! Take care!
